In [5]:
import os
import re
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from analysis.analyzer import TTTH_Analyzer
from processor.feature import FeatureProcessor
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
# from underthesea import word_tokenize, pos_tag, sent_tokenize
from pyvi.ViTokenizer import tokenize
from pyvi import ViTokenizer
from underthesea import word_tokenize
from gensim import corpora, models, similarities
from tqdm import tqdm
_analyzer = TTTH_Analyzer()
_processor = FeatureProcessor()
warnings.filterwarnings("ignore")

In [6]:
df = pd.read_excel('data_motobikes.xlsx')

In [7]:
data = df.drop(columns=['Địa chỉ','Tình trạng',
       'Chính sách bảo hành', 'Trọng lượng', 'Href'])
data.head(3)

,id,Tiêu đề,Giá,Khoảng giá min,Khoảng giá max,Mô tả chi tiết,Thương hiệu,Dòng xe,Năm đăng ký,Số Km đã đi,Loại xe,Dung tích xe,Xuất xứ
0,1,"Bán Vespa Sprint 125cc 2024 xanh dương, xe đẹp...",66.000.000 đ,72.53 tr,85.14 tr,Bán xe #Vespa Sprint 125cc. Mua mới tại #Topco...,Piaggio,Vespa,2024,14000,Tay ga,100 - 175 cc,Đang cập nhật
1,2,🔥🔥SH 150i Thắng ABS 2019 BSTP Chính Chủ,79.500.000 đ,62.76 tr,73.68 tr,"_Bán SH 150i Thắng ABS 2019 Xám Bạc, Úp Team X...",Honda,SH,2019,28000,Tay ga,100 - 175 cc,Đang cập nhật
2,3,CC Vision Thể Thao 2023 Đen+bộ đèn Demi audi A7,37.000.000 đ,28 tr,32.86 tr,Chính chủ bán Vision phiên bản Thể Thao 2023 Đ...,Honda,Vision,2023,12000,Tay ga,100 - 175 cc,Đang cập nhật


In [8]:
# Sắp xếp dữ liệu theo Thương hiệu, Dòng xe, Loại xe (tăng dần)
data = data.sort_values(by=['Thương hiệu', 'Dòng xe', 'Loại xe'], ascending=[True, True, True])
# Reset lại index sau khi sắp xếp
data = data.reset_index(drop=True)
###############################

#Chuẩn hóa cột "Giá"
data['Giá'] = (
    data['Giá']
    .astype(str)
    .str.replace(r'[^\d]', '', regex=True)  # loại bỏ mọi ký tự không phải số
)
# Đổi chuỗi rỗng thành NaN
data.loc[data['Giá'] == '', 'Giá'] = np.nan
# Ép kiểu float và chia cho 1,000,000 để ra đơn vị triệu
data['Giá'] = data['Giá'].astype(float) / 1_000_000 

for col in ['Khoảng giá min', 'Khoảng giá max']:
    data[col] = (
        data[col]
        .astype(str)
        .str.replace('tr', '', case=False, regex=False)  # bỏ chữ "tr"
        .str.replace(',', '.')  # nếu có dấu phẩy
        .str.strip()  # bỏ khoảng trắng
    )

    # Đổi chuỗi rỗng thành NaN rồi ép kiểu float
    data.loc[data[col] == '', col] = np.nan
    data[col] = data[col].astype(float)
###############################
data_clean = data.copy()
# 1. Xóa dòng thiếu tiêu đề hoặc giá
data_clean = data_clean.dropna(subset=['Tiêu đề', 'Giá'])

# 2. Điền khoảng giá min/max bằng cột Giá
data_clean['Khoảng giá min'] = data_clean['Khoảng giá min'].fillna(data_clean['Giá'])
data_clean['Khoảng giá max'] = data_clean['Khoảng giá max'].fillna(data_clean['Giá'])

# 3. Nếu vẫn còn NaN, điền median theo Thương hiệu
data_clean['Khoảng giá min'] = data_clean.groupby('Thương hiệu')['Khoảng giá min'].transform(
    lambda x: x.fillna(x.median())
)
data_clean['Khoảng giá max'] = data_clean.groupby('Thương hiệu')['Khoảng giá max'].transform(
    lambda x: x.fillna(x.median())
)
#############################################################

def price_segment(price):
    """
    Phân loại xe theo phân khúc giá.
    - Phổ thông: < 70 triệu
    - Cận cao cấp: 70–200 triệu
    - Cao cấp: > 200 triệu
    """
    if price < 70:
        return "Phổ thông"
    elif price < 200:
        return "Cận cao cấp"
    else:
        return "Cao cấp"

data_clean["Phân khúc giá"] = data_clean["Giá"].apply(price_segment)
##############################################################

# Chuyển về numeric
data_clean[['Giá', 'Khoảng giá min', 'Khoảng giá max']] = data_clean[
    ['Giá', 'Khoảng giá min', 'Khoảng giá max']
].astype(float)
# Lọc bỏ các giá bất thường
data_clean = data_clean[(data_clean['Giá'] > 1) & (data_clean['Giá'] < 5000)]
################################################################

# Xử lý cột Số Km đã đi
data_clean.loc[data_clean['Số Km đã đi'] > 99999, 'Số Km đã đi'] = 99999
################################################################


# Làm sạch text
for col in ['Thương hiệu', 'Dòng xe', 'Loại xe', 'Dung tích xe', 'Xuất xứ', 'Phân khúc giá']:
    data_clean[col] = data_clean[col].str.strip().str.title()

# Dung tích xe: map định lượng
def parse_cc(val):
    if 'Dưới' in val: return 40
    if '50 - 100' in val: return 75
    if '100 - 175' in val: return 137
    if 'Trên 175' in val: return 200
    return np.nan
data_clean['cc_numeric'] = data_clean['Dung tích xe'].apply(parse_cc)
######################################################################

# Phân khúc giá: map ordinal
price_segment_map = {'Phổ Thông': 1, 'Cận Cao Cấp': 2, 'Cao Cấp': 3}
data_clean['price_segment_code'] = data_clean['Phân khúc giá'].map(price_segment_map)
#######################################################################

# Thay các giá trị đặc biệt trong cột Năm đăng ký
data_clean['Năm đăng ký'] = data_clean['Năm đăng ký'].replace({
    'trước năm 1980': '1979',
    'Đang cập nhật': np.nan,
    'Không rõ': np.nan
})
# Chuyển sang kiểu int
data_clean['Năm đăng ký'] = pd.to_numeric(data_clean['Năm đăng ký'], errors='coerce')
data_clean['Năm đăng ký'] = data_clean['Năm đăng ký'].astype(int)

min_age = 0.5  # tính tròn cho 6 tháng
data_clean['age'] = 2025 - data_clean['Năm đăng ký']

# Thay age == 0 bằng min_age
data_clean.loc[data_clean['age'] <= 0, 'age'] = min_age
#######################################################################

# Xử lý missing values cho cột cc_numeric
_processor.handle_missing_values_by_median('cc_numeric', data_clean)

cc_numeric before fill missing values: 65
cc_numeric after fill missing values: 0


In [9]:
numeric_cols = [
    "Giá", "Khoảng giá min", "Khoảng giá max",
    "Số Km đã đi", "age", "cc_numeric"]


# Danh sách thương hiệu mô tô cao cấp
premium_brands = ['BMW', 'Harley Davidson', 'Ducati', 'Triumph', 'Kawasaki', 'Benelli']

# Áp dụng ngưỡng giá tối đa cho xe phổ thông
data_clean.loc[
    (~data_clean['Thương hiệu'].isin(premium_brands)) & (data_clean['Giá'] > 300),
    'Giá'
] = 300

# Phát hiện outliers sử dụng IQR
Q1 = data_clean[numeric_cols].quantile(0.25)
Q3 = data_clean[numeric_cols].quantile(0.75)
IQR = Q3 - Q1
outlier_mask = (data_clean[numeric_cols] < (Q1 - 1.5 * IQR)) | (data_clean[numeric_cols] > (Q3 + 1.5 * IQR))
outlier_counts = outlier_mask.sum().sort_values(ascending=False)

In [10]:
# Định nghĩa các nhóm keyword

# Nhóm 1: MỚI / TÌNH TRẠNG XE
kw_moi = [
    "mới", "còn mới", "như mới", "mới 95", "mới 99", "mới tinh",
    "xe lướt", "xe ít đi", "ít sử dụng", "xe để không", "để kho",
    "keng", "leng keng", "nguyên zin", "zin 100%", "zin nguyên bản",
    "dán keo", "dán ppf", "ngoại hình đẹp", "dàn áo liền lạc", "đẹp như hình"
]
# Nhóm 2: ĐỘ XE / ĐỒ CHƠI / NÂNG CẤP
kw_do_xe = [
    "độ", "đồ chơi", "full đồ", "pô độ", "pô móc", "phuộc rcb", "tay thắng",
    "lên đồ", "tem độ", "lên full đồ", "đồ zin còn đủ", "kính gió", "thùng givi",
    "ốc titan", "mão gió", "bao tay", "trợ lực", "độ máy"
]
# Nhóm 3: MỨC ĐỘ SỬ DỤNG
kw_su_dung = [
    "ít đi", "đi làm", "đi học", "đi phượt", "đi cà phê", "để không",
    "ít sử dụng", "xe gia đình", "xe công ty", "dư xe", "đi lại nhẹ nhàng",
    "xe nữ dùng", "xe nữ chạy", "xe để lâu", "ít chạy", "đi gần"
]
# Nhóm 4: BẢO DƯỠNG / SỬA CHỮA
kw_bao_duong = [
    "bảo dưỡng", "bảo trì", "thay nhớt", "vệ sinh", "bao test", "đi bảo dưỡng",
    "bảo dưỡng định kỳ", "mới thay bình", "mới làm nồi", "đã làm lại máy",
    "thay bố thắng", "thay lọc", "bảo dưỡng lớn", "chỉnh sên", "xe kỹ"
]
# Nhóm 5: ĐỘ BỀN / MÁY MÓC / CHẤT LƯỢNG
kw_do_ben = [
    "máy êm", "nổ êm", "chạy êm", "máy mạnh", "máy bốc", "tiết kiệm xăng",
    "ổn định", "chạy ngon", "không xì nhớt", "không rò rỉ", "không lỗi",
    "máy khô ráo", "máy tốt", "chạy mượt", "vận hành ổn định", "êm ái",
    "bền bỉ", "máy móc zin", "chạy bình thường", "hoạt động tốt"
]
# Nhóm 6: GIẤY TỜ / PHÁP LÝ
kw_phap_ly = [
    "chính chủ", "ủy quyền", "bao sang tên", "cà vẹt", "giấy tờ đầy đủ",
    "giấy tờ hợp lệ", "hồ sơ gốc", "bstp", "bao công chứng", 
    "bao tranh chấp", "ra tên", "cavet", "hợp pháp"
]


In [11]:
# Hàm check từ khóa xuất hiện trong mô tả
def keyword_flag(text: str, keywords: list[str]) -> int:
    """
    Kiểm tra xem text có chứa ít nhất 1 từ khóa trong danh sách không.
    Trả về 1 nếu có, 0 nếu không.
    """
    if pd.isna(text):
        return 0
    text = text.lower()
    return int(any(re.search(rf"(?<!\w){re.escape(kw)}(?!\w)", text) for kw in keywords))

# Làm sạch và chuẩn hóa văn bản
def clean_text(text: str) -> str:
    """
    Chuẩn hóa mô tả:
    - Chuyển về chữ thường
    - Bỏ URL, ký tự đặc biệt, số
    - Chuẩn hóa khoảng trắng
    """
    if pd.isna(text):
        return ""
    text = text.lower()
    text = re.sub(r"http\S+|www\S+", "", text)
    text = re.sub(r"\d+", "", text)
    text = re.sub(f"[{re.escape(string.punctuation)}]", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Loại bỏ stopwords tiếng Việt
vietnamese_stopwords = set([
    "xe", "máy", "bán", "cần", "mua", "báo", "liên", "hệ", "anh", "chị",
    "em", "mn", "mọi", "người", "xin", "cảm", "ơn", "chợ", "tốt", "đầy",
    "đủ", "điện", "thoại", "địa", "chỉ", "số", "của", "và", "với", "còn",
    "thì", "nên", "rất", "đã", "được", "ko", "kg", "thật", "là", "thôi",
    "nha", "nhé", "ạ", "nhưng", "bởi", "vì", "thì", "nào", "vậy"
])

def remove_stopwords(text: str) -> str:
    words = text.split()
    return " ".join([w for w in words if w not in vietnamese_stopwords])

In [12]:
# Áp dụng NLP
data_clean["desc_clean"] = data_clean["Mô tả chi tiết"].apply(clean_text)
data_clean["desc_clean"] = data_clean["desc_clean"].apply(remove_stopwords)


# Áp dụng tạo đặc trưng mới
data_clean["is_moi"] = data_clean["desc_clean"].apply(lambda x: keyword_flag(x, kw_moi))
data_clean["is_do_xe"] = data_clean["desc_clean"].apply(lambda x: keyword_flag(x, kw_do_xe))
data_clean["is_su_dung_nhieu"] = data_clean["desc_clean"].apply(lambda x: keyword_flag(x, kw_su_dung))
data_clean["is_bao_duong"] = data_clean["desc_clean"].apply(lambda x: keyword_flag(x, kw_bao_duong))
data_clean["is_do_ben"] = data_clean["desc_clean"].apply(lambda x: keyword_flag(x, kw_do_ben))
data_clean["is_phap_ly"] = data_clean["desc_clean"].apply(lambda x: keyword_flag(x, kw_phap_ly))

In [14]:
stop_word_file = 'files/vietnamese-stopwords.txt'
emojicon_file = 'files/emojicon.txt'
teencode_file = 'files/teencode.txt'

# Load stopwords, emojicons, teencode mappings
with open(stop_word_file, 'r', encoding='utf-8') as f:
    stopwords = set([w.strip() for w in f.readlines() if w.strip()])

with open(emojicon_file, 'r', encoding='utf-8') as f:
    emojicons = [w.strip() for w in f.readlines() if w.strip()]

with open(teencode_file, 'r', encoding='utf-8') as f:
    teencode_map = {}
    for line in f:
        parts = line.strip().split()
        if len(parts) >= 2:
            teencode_map[parts[0]] = " ".join(parts[1:])


special_tokens = ['', ' ', ',', '.', '...', '-', ':', ';', '?', '%', '(', ')', '+', '/', "'", '&', '#', '*', '!', '"', '_', '=', '[', ']', '{', '}', '~', '`', '|', '\\']


In [15]:
# Các hàm xử lý
def remove_emojis(text):
    for emo in emojicons:
        text = text.replace(emo, ' ')
    return text

def normalize_teencode(text):
    for key, val in teencode_map.items():
        text = re.sub(rf'\b{re.escape(key)}\b', val, text)
    return text

def remove_special_chars(text):
    text = re.sub(r'[^\w\s]', ' ', text)  # loại ký tự đặc biệt
    text = re.sub(r'\s+', ' ', text).strip()  # loại khoảng trắng thừa
    return text

# -----------------------
# 4. TÁCH STOPWORD RIÊNG
# -----------------------
def remove_stopwords(text):
    tokens = word_tokenize(text, format="text").split()
    tokens = [t for t in tokens if t not in stopwords]
    return ' '.join(tokens)

# -----------------------
# 5. CHUẨN HÓA TỔNG HỢP
# -----------------------
def clean_text(text):
    text = str(text).lower()
    text = remove_emojis(text)
    text = normalize_teencode(text)
    text = remove_special_chars(text)
    text = remove_stopwords(text)
    return text

In [16]:
data_clean['Content'] = data_clean['Mô tả chi tiết'].apply(lambda x: ' '.join(x.split()[:200]))

data_clean['clean_text'] = data_clean['Content'].apply(clean_text)

In [17]:
data_clean.columns

Index(['id', 'Tiêu đề', 'Giá', 'Khoảng giá min', 'Khoảng giá max',
       'Mô tả chi tiết', 'Thương hiệu', 'Dòng xe', 'Năm đăng ký',
       'Số Km đã đi', 'Loại xe', 'Dung tích xe', 'Xuất xứ', 'Phân khúc giá',
       'cc_numeric', 'price_segment_code', 'age', 'desc_clean', 'is_moi',
       'is_do_xe', 'is_su_dung_nhieu', 'is_bao_duong', 'is_do_ben',
       'is_phap_ly', 'Content', 'clean_text'],
      dtype='object')

In [18]:
vectorizer = TfidfVectorizer(
    analyzer='word',
    max_features=8000
)
tfidf_matrix = vectorizer.fit_transform(data_clean['clean_text'])
cosine_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

def recommend(item_id: int, top_n: int = 5):
    """
    Recommend similar motorbikes based on cosine similarity.
    Args:
        item_id (int): id hoặc index của xe trong DataFrame
        top_n (int): số lượng gợi ý muốn lấy
    Returns:
        DataFrame chứa các xe tương tự
    """
    if item_id not in data.index:
        raise ValueError(f"id {item_id} không tồn tại trong DataFrame")

    # Lấy hàng tương ứng trong ma trận cosine
    sim_scores = list(enumerate(cosine_sim_matrix[item_id]))

    # Sắp xếp theo độ tương đồng giảm dần, bỏ chính nó
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1: top_n + 1]

    # Lấy index xe tương tự
    similar_indices = [i[0] for i in sim_scores]
    similar_scores = [i[1] for i in sim_scores]

    # Tạo DataFrame kết quả
    recommendations = data.loc[similar_indices, ['id', 'Tiêu đề', 'Content']].copy()
    recommendations['similarity'] = similar_scores
    return recommendations.reset_index(drop=True)

def recommend_cosine_by_text(query: str, top_n: int = 5):
    """
    Gợi ý xe máy tương tự dựa trên văn bản người dùng nhập vào.
    
    Args:
        query (str): văn bản tìm kiếm
        top_n (int): số lượng gợi ý
    
    Returns:
        DataFrame: danh sách xe tương tự + độ tương đồng
    """

    # 1. Tiền xử lý query bằng hàm clean_text của bạn
    clean_query = clean_text(query)

    # 2. Vector hóa query
    query_vec = vectorizer.transform([clean_query])

    # 3. Tính độ tương đồng cosine giữa query và toàn bộ item
    sims = cosine_similarity(query_vec, tfidf_matrix).flatten()

    # 4. Lấy top N kết quả cao nhất
    top_idx = sims.argsort()[::-1][:top_n]
    top_scores = sims[top_idx]

    # 5. Trả về DataFrame kết quả
    result = data.iloc[top_idx][['id', 'Tiêu đề', 'Content']].copy()
    result["similarity"] = top_scores

    return result.reset_index(drop=True)
